In [ ]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from plotly import express as px
from typing import List, Tuple
import pickle

In [ ]:
import sys
sys.path.append('../../../../')

## load

In [ ]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

In [ ]:
fold = folds_training_dicts[0]
fold.keys()

In [ ]:
input_data = fold['components_train']
coordinates = fold['coordinates_train']
size_value = fold['sizes_train']
n_patches_hot_encoded_value = fold['num_patches_train']
max_number_of_patches = 10

## debug pairwise transition

In [ ]:
from model import create_masked_inputs

features, pairwise_distances = create_masked_inputs(
        input_data, coordinates, size_value, n_patches_hot_encoded_value, max_number_of_patches)

In [ ]:
px.imshow(pairwise_distances[0][:, :, 0])

In [ ]:
px.imshow(pairwise_distances._keras_mask[0])